# B1 interactive filtering

In [ ]:
cd qMRLab
startup

In [ ]:
cmd = ['curl -L -o b1_map.zip https://osf.io/mw3sq/download/'];
[STATUS,MESSAGE] = unix(cmd);
unzip('b1_map.zip');

In [ ]:
clear all

Model = b1_dam;

% Format data structure so that they may be fit by the model
data = struct();
data.SFalpha = double(load_nii_data('SFalpha.nii.gz')); %load data
data.SF2alpha  = double(load_nii_data('SF2alpha.nii.gz'));

Model.options.Smoothingfilter_Type = 'spline';
Model.options.Smoothingfilter_order = 2;
FitResults_spline = FitData(data,Model);

In [ ]:
B1_raw = FitResults_spline.B1map_raw;
B1_filtered = FitResults_spline.B1map_filtered;

In [ ]:
%get B1_raw --from Octave 
%get B1_filtered --from Octave 
%matplotlib inline

import matplotlib.pyplot as mpl
from matplotlib import cm
import numpy as np

mpl.imshow(np.concatenate((B1_raw, B1_filtered), axis=1))
mpl.clim(0.9,1.05)
mpl.colorbar()
mpl.set_cmap('jet')